# Load Models and Generate the submission File

### This notebook will show how to
- load the checkpoints of our baseline models
- use our API to generate a submission .csv file
- use the .csv file to make a submission

We also show how
- we extract the ground truth responses from a dataset
- we use these ground truth responses and the submitted files to calculate the score of the live leaderboard of our competition

In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

In [2]:
tier = 'train'
dataset_name = '20457-5-9-0'

filenames = ['./data/lurz2020/static20457-5-9-preproc0', ]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': True,
                 'batch_size': 128,
                 'exclude': None,
                 'file_tree': True,
                 'scale': 1,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

In [3]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
              'stack': -1,
              'layers': 4,
              'input_kern': 9,
              'gamma_input': 6.3831,
              'gamma_readout': 0.0076,
              'hidden_dilation': 1,
              'hidden_kern': 7,
              'hidden_channels': 64,
              'depth_separable': True,
              'init_sigma': 0.1,
              'init_mu_range': 0.3,
              'gauss_type': 'full',
               }

sota_model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

sota_model.load_state_dict(torch.load("./checkpoints/sota_model.pth"));

In [4]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
              'stack': -1,
              'layers': 3,
              'input_kern': 9,
              'gamma_input': 6.3831,
              'gamma_readout': 0.0076,
              'hidden_dilation': 1,
              'hidden_kern': 7,
              'hidden_channels': 64,
              'depth_separable': True,
              'init_sigma': 0.1,
              'init_mu_range': 0.3,
              'gauss_type': 'full',
              'linear': True
               }
ln_model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

ln_model.load_state_dict(torch.load("./checkpoints/ln_model.pth"));

# Generate the Submission File with our API

In [5]:
# import the API from the competition repo
from sensorium.utility import submission

In [6]:
# generate the submission file
submission.generate_submission_file(trained_model=sota_model, 
                                    test_dataloader=dataloaders["test"][dataset_name], 
                                    path='./submission_files/sota_submission_file.csv')

File saved.


In [7]:
# identical procedure with the LN model
submission.generate_submission_file(trained_model=ln_model, 
                                    test_dataloader=dataloaders["test"][dataset_name], 
                                    path='./submission_files/ln_submission_file.csv')

File saved.


### Inspect content of submission file

In [8]:
import pandas as pd
pd.read_csv('./submission_files/sota_submission_file.csv')

,trial_indices,image_ids,prediction,neuron_ids
0,126,2214,"[0.40991318225860596, 0.9949695467948914, 0.53...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
1,297,2214,"[0.40917062759399414, 0.9313706755638123, 0.56...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
2,597,2214,"[0.39792776107788086, 0.9711095690727234, 0.50...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
3,852,2214,"[0.40897947549819946, 0.9369102120399475, 0.55...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
4,908,2214,"[0.4109222888946533, 0.9365361928939819, 0.517...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
...,...,...,...,...
994,2752,3487,"[0.14154642820358276, 0.17261016368865967, 0.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
995,3039,3487,"[0.1326291561126709, 0.16848045587539673, 0.28...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
996,4312,3487,"[0.1303417682647705, 0.15538263320922852, 0.28...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
997,4380,3487,"[0.14169669151306152, 0.20502185821533203, 0.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."


In [9]:
import pandas as pd
pd.read_csv('./submission_files/ln_submission_file.csv')

,trial_indices,image_ids,prediction,neuron_ids
0,126,2214,"[0.2693312168121338, 0.4970896244049072, 0.495...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
1,297,2214,"[0.2529667019844055, 0.5307848453521729, 0.497...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
2,597,2214,"[0.27215510606765747, 0.522422730922699, 0.497...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
3,852,2214,"[0.2690316438674927, 0.5220757722854614, 0.502...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
4,908,2214,"[0.25832074880599976, 0.531325101852417, 0.460...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
...,...,...,...,...
994,2752,3487,"[0.2589794397354126, 0.8648760318756104, 0.975...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
995,3039,3487,"[0.2580360174179077, 0.8084425926208496, 1.059...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
996,4312,3487,"[0.2702764868736267, 0.7732750773429871, 1.034...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
997,4380,3487,"[0.25859391689300537, 0.9139286279678345, 0.99...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."


# The submission process

- now that the .csv file is generated, the participant can make an entry to the competition
- all that is left to do is:
 - register a new user on http://sensorium2022.net/ with a valid email address
 - upload the .csv file
 - the competition score will be calculated automatically and will show up in the live leaderboard within a few seconds

---

---

# How we calculate the scores

- we will withhold the ground truth neuronal responses to the test set images in the actual competition
- here we show 
 - how we extract the ground truth responses from the demo dataset (where the test set responses are present)
 - how the metrics of the competition are calculated

In [10]:
from sensorium.eval import evaluate

In [11]:
filename = ['./data/lurz2020/static20457-5-9-preproc0']
submission.generate_ground_truth_file(filename=filename,
                                      path='./ground_truth_files/ground_truth_file.csv')

### SOTA model results

In [12]:
ground_truth_file = './ground_truth_files/ground_truth_file.csv'
submission_file = './submission_files/sota_submission_file.csv'

In [13]:
out = evaluate(submission_file, ground_truth_file)

In [14]:
print("Results for the SOTA model:")
for metric, value in out.items():
    print(f"{metric}: {np.round(value, 3)}")

Results for the SOTA model:
Correlation (single trial): 0.297
Correlation (mean): 0.542
FEVE: 0.303


#### These scores are calcualted in the backend of our website
- we have two test sets, so these scores will be computed for our **live** test set, and our **final** test set
- the **live** scores will get published on the live leaderboard
- the **final** scores will be hidden, and we will release them to the public on Oct 22, after checking the scores carefully
- the **final** scores will then determine the winner of the competition

## Same process for the simple Baseline Model

### LN model results

In [15]:
ground_truth_file = './ground_truth_files/ground_truth_file.csv'
submission_file = './submission_files/ln_submission_file.csv'

In [16]:
out = evaluate(submission_file, ground_truth_file)

In [17]:
print("Results for the LN model:")
for metric, value in out.items():
    print(f"{metric}: {np.round(value,3)}")

Results for the LN model:
Correlation (single trial): 0.203
Correlation (mean): 0.365
FEVE: 0.13


---